<a href="https://colab.research.google.com/github/tuananh1006/Web-Scraping-Book/blob/main/CrawlerBase.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from urllib.request import urlopen
from bs4 import BeautifulSoup

In [2]:
class Content:
  def __init__(self,url,title,body):
    self.url=url
    self.title=title
    self.body=body
  def print(self):
    print("title:"+self.title)
    print("url:"+self.url)
    print("Body:"+self.body)



In [3]:
def scrapeCNN(url):
    bs = BeautifulSoup(urlopen(url))
    title = bs.find('h1').text
    body = bs.find('div', {'class': 'article__content'}).text
    print('body: ')
    print(body)
    return Content(url, title, body)

In [39]:
def scrapeBrookings(url):
    bs = BeautifulSoup(urlopen(url))
    title = bs.find('title').text
    content=[]
    for description in bs.find_all('p', {'class': ['description','Normal']}):
      content.append(description.get_text().replace('\n',''))
    body="\n".join(content)
    return Content(url, title, body)


In [49]:
url = 'https://vnexpress.net/israel-yeu-cau-dan-thuong-so-tan-khoi-25-khu-vuc-o-lebanon-4799966.html'


In [41]:
content = scrapeBrookings(url)
content.print()

title:Israel yêu cầu dân thường sơ tán khỏi 25 khu vực ở Lebanon - Báo VnExpress
url:https://vnexpress.net/israel-yeu-cau-dan-thuong-so-tan-khoi-25-khu-vuc-o-lebanon-4799966.html
Body:Quân đội Israel phát cảnh báo đến người dân ở 25 địa phương tại miền nam Lebanon, kêu gọi họ lập tức sơ tán để đảm bảo tính mạng.
"Hành động của Hezbollah khiến Lực lượng Phòng vệ Israel (IDF) phải đối phó. Chúng tôi không có ý định làm hại quý vị. Vì an toàn của bản thân, quý vị phải lập tức sơ tán", đại tá Avichay Adraee, phát ngôn viên tiếng Arab của IDF, thông báo hôm nay.
IDF cũng công bố danh sách 25 thành phố, thị trấn và làng mạc ở miền nam Lebanon mà dân thường cần di tản, trong đó có Nabatieh, đô thị lớn hàng đầu khu vực.
"Bất cứ ai ở gần thành viên, cơ sở hạ tầng và vũ khí của Hezbollah đều tự đặt mình vào vòng nguy hiểm", ông Adraee cho biết thêm, kêu gọi người dân Lebanon di chuyển lên phía bắc và cho biết sẽ cập nhật thông tin về thời điểm họ được trở về nhà sau.
IDF ngày 2/10 cũng đưa ra cả

In [43]:
class Website:
    """
    Contains information about website structure
    """
    def __init__(self, name, url, titleTag, bodyTag):
        self.name = name
        self.url = url
        self.titleTag = titleTag
        self.bodyTag = bodyTag

In [73]:
class Crawler:
    def getPage(url):
        try:
            html = urlopen(url)
        except Exception:
            return None
        return BeautifulSoup(html, 'html.parser')

    def safeGet(bs, selector):
        """
        Utility function used to get a content string from a Beautiful Soup
        object and a selector. Returns an empty string if no object
        is found for the given selector
        """
        selectedElems = bs.select(selector)
        if selectedElems is not None and len(selectedElems) > 0:
            return '\n'.join([elem.get_text().replace('\n','') for elem in selectedElems])
        return ''
    def getContent(website, path):
        """
        Extract content from a given page URL
        """
        url = website.url+path
        bs = Crawler.getPage(url)
        if bs is not None:
            title = Crawler.safeGet(bs, website.titleTag)
            body = Crawler.safeGet(bs, website.bodyTag)
            return Content(url, title, body)
        return Content(url, '', '')

In [74]:
siteData = [
    ['O\'Reilly', 'https://www.oreilly.com', 'h1', 'div.title-description'],
    ['Reuters', 'https://www.reuters.com', 'h1', 'div.ArticleBodyWrapper'],
    ['Brookings', 'https://www.brookings.edu', 'h1', 'div.post-body'],
    ['CNN', 'https://www.cnn.com', 'h1', 'div.article__content']
]
websites = []
for name, url, title, body in siteData:
    websites.append(Website(name, url, title, body))

Crawler.getContent(
    websites[0],
    '/library/view/web-scraping-with/9781491910283'
    ).print()
Crawler.getContent(
    websites[1],
    '/article/us-usa-epa-pruitt-idUSKBN19W2D0'
    ).print()
Crawler.getContent(
    websites[2],
    '/blog/techtank/2016/03/01/idea-to-retire-old-methods-of-policy-education/').print()
Crawler.getContent(
    websites[3],
    '/2023/04/03/investing/dogecoin-elon-musk-twitter/index.html'
    ).print()

title:Web Scraping with Python
url:https://www.oreilly.com/library/view/web-scraping-with/9781491910283
Body:              Book      descriptionLearn web scraping and crawling techniques to access unlimited data from any web source in any format. With this practical guide, you’ll learn how to use Python scripts and web APIs to gather and process data from thousands—or even millions—of web pages at once.Ideal for programmers, security professionals, and web administrators familiar with Python, this book not only teaches basic web scraping mechanics, but also delves into more advanced topics, such as analyzing raw data or using scrapers for frontend website testing. Code samples are available to help you understand the concepts in practice.Show and hide morePublisher resourcesView/Submit Errata
title:
url:https://www.reuters.com/article/us-usa-epa-pruitt-idUSKBN19W2D0
Body:
title:
url:https://www.brookings.edu/blog/techtank/2016/03/01/idea-to-retire-old-methods-of-policy-education/
Body:

In [75]:
class Content:
    """Common base class for all articles/pages"""

    def __init__(self, topic, url, title, body):
        self.topic = topic
        self.title = title
        self.body = body
        self.url = url

    def print(self):
        """
        Flexible printing function controls output
        """
        print('New article found for topic: {}'.format(self.topic))
        print('URL: {}'.format(self.url))
        print('TITLE: {}'.format(self.title))
        print('BODY:\n{}'.format(self.body))

In [77]:
class Website:
    """Contains information about website structure"""

    def __init__(self, name, url, searchUrl, resultListing,resultUrl, absoluteUrl, titleTag, bodyTag):
        self.name = name
        self.url = url
        self.searchUrl = searchUrl
        self.resultListing = resultListing
        self.resultUrl = resultUrl
        self.absoluteUrl=absoluteUrl
        self.titleTag = titleTag
        self.bodyTag = bodyTag

In [80]:
class Crawler:
    def __init__(self, website):
        self.site = website
        self.found = {}

    def getPage(url):
        try:
            html = urlopen(url)
        except Exception:
            return None
        return BeautifulSoup(html, 'html.parser')

    def safeGet(bs, selector):
        """
        Utility function used to get a content string from a Beautiful Soup
        object and a selector. Returns an empty string if no object
        is found for the given selector
        """
        selectedElems = bs.select(selector)
        if selectedElems is not None and len(selectedElems) > 0:
            return '\n'.join([elem.get_text().replace('\n','') for elem in selectedElems])

    def getContent(self, topic, url):
        """
        Extract content from a given page URL
        """
        bs = Crawler.getPage(url)
        if bs is not None:
            title = Crawler.safeGet(bs, self.site.titleTag)
            body = Crawler.safeGet(bs, self.site.bodyTag)
            return Content(topic, url, title, body)
        return Content(topic, url, '', '')

    def search(self, topic):
        """
        Searches a given website for a given topic and records all pages found
        """
        bs = Crawler.getPage(self.site.searchUrl + topic)
        searchResults = bs.select(self.site.resultListing)
        for result in searchResults:
            url = result.select(self.site.resultUrl)[0].attrs['href']
            # Check to see whether it's a relative or an absolute URL
            url = url if self.site.absoluteUrl else self.site.url + url
            if url not in self.found:
                self.found[url] = self.getContent(topic, url)
            self.found[url].print()

In [ ]:
siteData = [
    ['Reuters', 'http://reuters.com',
     'https://www.reuters.com/search/news?blob=',
     'div.search-result-indiv', 'h3.search-result-title a',
      False, 'h1', 'div.ArticleBodyWrapper'],
    ['Brookings', 'http://www.brookings.edu',
     'https://www.brookings.edu/search/?s=',
        'div.article-info', 'h4.title a', True, 'h1', 'div.core-block']
]
sites = []
for name, url, search, rListing, rUrl, absUrl, tt, bt in siteData:
    sites.append(Website(name, url, search, rListing, rUrl, absUrl, tt, bt))

crawlers = [Crawler(site) for site in sites]
topics = ['python', 'data%20science']

for topic in topics:
    for crawler in crawlers:
        crawler.search(topic)

In [89]:
class Website:
    def __init__(self, name, url, targetPattern, absoluteUrl, titleTag, bodyTag):
        self.name = name
        self.url = url
        self.targetPattern = targetPattern
        self.absoluteUrl = absoluteUrl
        self.titleTag = titleTag
        self.bodyTag = bodyTag

class Content:
    def __init__(self, url, title, body):
        self.url = url
        self.title = title
        self.body = body

    def print(self):
        print(f'URL: {self.url}')
        print(f'TITLE: {self.title}')
        print(f'BODY:\n{self.body}')

In [ ]:
class Crawler:
    def __init__(self, site):
      self.site = site
      self.visited = {}

    def getPage(url):
      try:
            html = urlopen(url)
      except Exception as e:
            print(e)
            return None
      return BeautifulSoup(html, 'html.parser')

    def safeGet(bs, selector):
      selectedElems = bs.select(selector)
      if selectedElems is not None and len(selectedElems) > 0:
            return '\n'.join([elem.get_text() for elem in selectedElems])
      return ''

    def getContent(self, url):
      """
      Extract content from a given page URL
      """
      bs = Crawler.getPage(url)
      if bs is not None:
          title = Crawler.safeGet(bs, self.site.titleTag)
          body = Crawler.safeGet(bs, self.site.bodyTag)
          return Content(url, title, body)
        return Content(url, '', '')

    def crawl(self):
        """
        Get pages from website home page
        """
        bs = Crawler.getPage(self.site.url)
        targetPages = bs.findAll('a', href=re.compile(self.site.targetPattern))
        for targetPage in targetPages:
          url = targetPage.attrs['href']
          url = url if self.site.absoluteUrl else f'{self.site.url}{targetPage}'
          if url not in self.visited:
                self.visited[url] = self.getContent(url)
                self.visited[url].print()

brookings = Website(
    'Brookings', 'https://brookings.edu', '\/(research|blog)\/',
     True, 'h1', 'div.post-body')
crawler = Crawler(brookings)
crawler.crawl()

In [92]:
class Website:
    def __init__(self, name, url, titleTag, bodyTag, pageType):
        self.name = name
        self.url = url
        self.titleTag = titleTag
        self.bodyTag = bodyTag
        self.pageType = pageType

In [93]:
class Product(Website):
    """Contains information for scraping a product page"""
    def __init__(self, name, url, titleTag, productNumberTag, priceTag):
        Website.__init__(self, name, url, TitleTag)
        self.productNumberTag = productNumberTag
        self.priceTag = priceTag

class Article(Website):
    """Contains information for scraping an article page"""
    def __init__(self, name, url, titleTag, bodyTag, dateTag):
        Website.__init__(self, name, url, titleTag)
        self.bodyTag = bodyTag
        self.dateTag = dateTag